## Building a Reflection Agent using Strands Agent SDK

This notebook demonstrates how to build a multi-agent system using Amazon Bedrock and the Strands framework. The system consists of specialized agents that work together to create and refine PR (Public Relations) articles for movies or shows. The notebook showcases agent collaboration, knowledge base integration, and reflection capabilities.

To learn more about Strands Agent SDK, please follow this [link](https://strandsagents.com/latest/)

Let's get started!

In [ ]:
%pip install strands-agents strands-agents-tools -Uq

## Environment Setup

### Loading Environment Variables
setting up logger, initiates strands Agent libraries

<div class="alert alert-info"> 💡 The following cell imports Strands agent packages into the kernel. Sometimes, the library might not be updated from the installation. If you run into any issues with importing the following libraries, simply restart the kernel again and rerun the notebook. 
</div>

In [ ]:
from strands import Agent, tool
from strands_tools import retrieve

In [ ]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()])

## AWS Service Setup

In [ ]:
import boto3

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

bedrock_client = boto3.client('bedrock-runtime', region)

## Restore variables from previous lab content

In [ ]:
%store -r

## PR Writer Agent Implementation
The following code block creates a specialized agent for writing PR articles using a foundation model on Bedrock. 
Strands Agent SDK provides a @tool decorator to make it easy for developers to create tools. 
Detailed descriptions about the tool is embedded in the function description. The description is used by Strands Agent to determine the right tool to invoke within the agent loop. For more information about how to create tools, please refer to this [link](https://strandsagents.com/latest/user-guide/concepts/tools/tools_overview/)


In [ ]:
@tool
def pr_writer_agent(query: str) -> str:
    """
    Writes a public relation article (PR) content based on the instructions given in the query.

    Use this tool when you need to write a PR article given an outline, or rewriting an article to incorporate the feedback from a reviewer. 
    
    Args:
        query: the instructions to write the PR article.

    Returns:
        A PR article in markdown format.
    """
    print("PR Writer agent...")
    
    pr_writer = Agent(
        model="us.amazon.nova-pro-v1:0",
        system_prompt=f"""You are to write a PR article for an upcoming show/movie based on the user request, and an optional recommended changes.

When writing the PR article, you should follow the following guidelines:

1. Headline: Capture Attention & Key Details - Be Concise & Informative: Use active verbs like "announces," "debuts," or "unveils." Include the title, release date/platform, and a hook (e.g., talent or franchise ties).
Example: "A24 Reveals Trailer for Horror Thriller ‘Nightfall’ Starring Florence Pugh, Premiering October 2024."
Highlight Exclusives: If applicable, note premieres at festivals (e.g., "Sundance 2025 Official Selection").

2. Lead Paragraph: The 5 Ws - Summarize Who (studio, talent), What (title, genre), When (release date), Where (platforms/theaters), and Why (unique angle, legacy, or cultural relevance).
Example: "Marvel Studios’ ‘Avengers: Legacy,’ directed by Ryan Coogler, will premiere in theaters globally on May 3, 2025, marking the MCU’s first reboot of the iconic franchise."

3. Key Content Elements - Synopsis: Offer a 1–2 paragraph teaser without spoilers. Emphasize uniqueness (e.g., "a dystopian love story set in 2140’s AI-dominated society").
Talent & Production Credentials: Highlight A-list actors, acclaimed directors, or award-winning crews. Mention prior successes (e.g., "From the Oscar-winning producer of Parasite").
Behind-the-Scenes (BTS) Insights: Share filming locations, technical innovations, or adaptations (e.g., "Shot in Iceland using cutting-edge VR cinematography").

You also have access to a knowledge base tool that contains pristine, high quality PRs that have been published in the past. You should use the knowledge base in assisting with the writing.
The knowledge base ID you have acess to is {lab5_kb_id} and region is {region}.


Finally, the article content should be in markdown format. Put the entire article content in a <article> XML tag. Do not provide any other text other than the article. 
""",
        tools=[retrieve],
        callback_handler=None
    )
    return str(pr_writer(query))

Testing the PR Writer Agent
The following code block tests the PR writer agent with a sample query.


In [ ]:
query = """Write a PR news for an upcoming movie given the following information: 

Title: Midnight Vendetta
Tagline: “When the clock strikes twelve, justice wears no mask.”
Genre: Action/Thriller
Release Date: November 22, 2025 (Theatrical & IMAX)

Logline:
A disgraced former MI6 agent infiltrates a glittering Dubai masquerade ball to dismantle a trillion-dollar cyberweapons syndicate—but must confront his deadliest enemy: the traitor who framed him for murder.

Key Production/Cast Details:
Director: David Leitch (Atomic Blonde, Bullet Train)—promises “brutally elegant fight choreography blending Bourne-style close combat with Dubai’s opulent settings.”
Studio: Pika Pictures (Budget: $200M)
Filming Locations: Dubai’s Burj Khalifa, Palm Jumeirah, and a custom-built 360-degree rotating ballroom set for the climactic fight.
Soundtrack: Pulse-pounding score by Ludwig Göransson 
"""

Calls the PR writer agent with the sample query and prints the generated PR article.

In [ ]:
pr_writer_results = pr_writer_agent(query)
print(pr_writer_results)

## PR Review Agent Implementation
The following code block creates another specialized Strands agent for reviewing PR articles, providing feedback based on clarity, specificity, technical accessibility, and other quality metrics.


In [ ]:
@tool
def pr_review_agent(article: str) -> str:
    """
    Review a public relation article (PR) and provide feedback.

    Use this tool when you need to review a PR article given a fully written PR article for improvement.

    Args:
        article: the article to be rviewed.

    Returns:
        Feedback about the article.
    """
    
    print("PR Review agent...")
    
    pr_review_agent = Agent(
        model="us.amazon.nova-premier-v1:0",
        system_prompt="""You are an AI assistant that performs a review for a given PR article and provide any feedback to improve the quality. 
As a PR article reviewer, you are given the specific guidelines to evaluate the quality of the document.

1. Write in clear, crisp sentences." - long or confusing sentences as "clunky" when they're difficult for readers to understand.

Here are the tips on to identify unclear sentences:

- Sentences longer than 40 words or spanning more than two lines
- Sentences that require multiple readings to understand
- Sentences with repeated words, especially "and" and "to"

2. Be specific. Don't leave anything open to interpretation or leave your reader guessing. 

Here are some clues your sentence isn't specific enough: 
- it contains qualitative or subjective adjectives and adverbs (like "might," "often," or "may be"), 
- it leaves the reader asking questions.

Finally, provide your feedback in a <feedback> XML tag. Return only the feedback and nothing else.
""",
callback_handler=None)
    return str(pr_review_agent(article))
    

In the following, we test the PR review agent by passing result from the PR article generated by the writer agent, and prints the feedback.

In [ ]:
review_result = pr_review_agent(pr_writer_results)
print(review_result)

## PR Supervisor Agent Implementation
Putting everything together. The following code block creates a supervisor agent that coordinates between the PR writer and reviewer agents to iteratively improve the PR article through multiple rounds of feedback.


In [ ]:
pr_supervisor_agent = Agent(
        model="us.amazon.nova-premier-v1:0",
        system_prompt=f"""You are a supervisor AI agent. You are given 2 agents capable of following tasks:

1. PR Writer agent - Writes a draft of PR article based on the given instructions and/or recommendations.

2. PR Reviewer agent - Reviews the PR article generated by the PR Writer agent to provide recommendation to improve the PR article.

You are to leverage both the agents to work collaboratively and iteratively to arrive at the best version of the PR article. 

Here are the guidelines:
- You must not modify the article written by the PR writer. When a review of an article is needed, you must send the entire article content the the PR writer as it is including the <article> XML tag, along with the feedback from the PR reviewer including the <feedback> XML tag. These XML tags will provide the entire context the PR writer to know what is the article to be rewritten, and the feedback to incorporate.
- You should only iterate the draft and review iterations for maximum of 1 time before returning the final version of the PR article to the user. 

The final draft of the PR must be a complete version formatted in markdown in the final response. Do not provide any explanation in the final response, return only the final PR article in markdown format.""",
        tools=[pr_writer_agent, pr_review_agent],
        callback_handler=None
    )

## Running the Multi-Agent Workflow
Runs the complete multi-agent workflow by passing the movie information to the supervisor agent, which coordinates the writing and review process.

In [ ]:
query = """Write a PR news for an upcoming movie given the following information: 
Title: Midnight Vendetta
Tagline: “When the clock strikes twelve, justice wears no mask.”
Genre: Action/Thriller
Release Date: November 22, 2025 (Theatrical & IMAX)

Logline:
A disgraced former MI6 agent infiltrates a glittering Dubai masquerade ball to dismantle a trillion-dollar cyberweapons syndicate—but must confront his deadliest enemy: the traitor who framed him for murder.

Key Production/Cast Details:
Director: David Leitch (Atomic Blonde, Bullet Train)—promises “brutally elegant fight choreography blending Bourne-style close combat with Dubai’s opulent settings.”
Studio: Pika Pictures (Budget: $200M)
Filming Locations: Dubai’s Burj Khalifa, Palm Jumeirah, and a custom-built 360-degree rotating ballroom set for the climactic fight.
Soundtrack: Pulse-pounding score by Ludwig Göransson"""

pr_final_result = pr_supervisor_agent(query)

Prints the final PR article after multiple rounds of writing and review.

In [ ]:
print(str(pr_final_result))

## Session and State
Strands agents maintain conversation context, handle state management, and support persistent sessions across interactions.

Strands agents maintain state in several forms:

- Conversation History: The sequence of messages between the user and the agent
- Tool State: Information about tool executions and results
- Request State: Contextual information maintained within a single request

The following code block prints the conversation history for the agent interactions processed in the previous step.

In [ ]:
pr_supervisor_agent.messages